# Imports

In [5]:
# Reguläre Python Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Sklearn Imports, um den Datensatz in Trainings- und Testdaten zu splitten
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Import Logistic Regression model
from sklearn.linear_model import LogisticRegression

# Import Decision Tree model
from sklearn.tree import DecisionTreeClassifier

# Import Metriken für die Leistungsbewertung der Modelle
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# import confusion matrix plot function
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Random State for reproducibility
RANDOM_STATE = 42

# Testdata
TEST_RATIO = 0.3

# Load Data, Prepare Training- and Test Data and Scale Data

In [6]:
# Lade die Daten
df = pd.read_csv("../data/dataset_cleaned.csv")

# Teile den Datensatz auf in: feature set and Zielvariable (target label)
X = df.drop(['Churn', 'CLV_Continuous'], axis=1)  # Alle Spalten außer Churn und CLV_Continuous
y = df['Churn']                                   # Target (0, 1)

# Teile den Datensatz in Trainings- und Testdaten auf. (30% Testdaten)
# Hinweis: stratify für nahezu gleichbleibende Klassenverteilung
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_RATIO, random_state=RANDOM_STATE, stratify=y)

# scale the data
scaler = StandardScaler()
scaler.fit(X_train) # fit the Scaler only with training data, so that the test data does not influence the Scaler
X_train_scaled = scaler.transform(X_train) 
X_test_scaled = scaler.transform(X_test) #Test data is also scaled, but the Scaler is only fitted with the training data
pd.DataFrame(X_train_scaled).head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.854668,-0.297132,1.023164,0.857250,-1.009883,0.642340,-1.103596,-0.012083,-0.735548,0.029336,...,-0.527799,-0.455288,-0.485599,0.0,-0.536769,1.728187,0.877045,-0.57606,-0.579674,1.722807
1,-1.950143,0.380904,-0.710303,0.158637,-1.009883,0.642340,0.906128,1.136269,-0.148728,-0.579224,...,-0.777239,2.196410,-0.485599,0.0,1.862999,-0.578641,-1.140192,-0.57606,-0.579674,1.722807
2,0.174262,0.671490,-0.710303,-1.151670,0.874642,-1.556808,-1.103596,1.167774,-0.912263,0.029336,...,0.428951,-0.455288,-0.485599,0.0,-0.536769,1.728187,0.877045,-0.57606,-0.579674,-0.580448
3,-0.861492,-0.587718,-1.750384,1.194415,-1.009883,0.642340,-1.103596,1.632436,-0.875871,0.637896,...,-0.611241,-0.455288,-0.485599,0.0,-0.536769,-0.578641,-1.140192,-0.57606,-0.579674,-0.580448
4,-1.020152,-1.556341,1.023164,-1.170998,0.874642,-1.556808,0.906128,0.789302,-0.749791,-0.883504,...,-0.694246,-0.455288,-0.485599,0.0,-0.536769,-0.578641,-1.140192,-0.57606,1.725109,-0.580448


# Optimize the Selected Model

#### Anwendung von SMOTE (Oversampling-Verfahren)


SMOTE (Synthetic Minority Over-sampling Technique) ist eine Methode zur Behandlung von Klassenungleichgewichten, bei der künstliche neue Beispiele der Minderheitsklasse erzeugt werden, indem neue Datenpunkte zwischen bestehenden Minderheitsklasseninstanzen interpoliert werden. Dadurch wird die Minderheitsklasse künstlich vergrößert, was dem Modell hilft, diese besser zu erkennen.

In [7]:
# Import SMOTE for handling class imbalance
from imblearn.over_sampling import SMOTE

# Apply SMOTE to the training data
smote = SMOTE(random_state=RANDOM_STATE)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train) # type: ignore

print("Vor SMOTE:", X_train.shape, y_train.value_counts())
print("Nach SMOTE:", X_train_res.shape, pd.Series(y_train_res).value_counts()) # type: ignore

Vor SMOTE: (5964, 23) Churn
0    5527
1     437
Name: count, dtype: int64
Nach SMOTE: (11054, 23) Churn
0    5527
1    5527
Name: count, dtype: int64


Die Anwendung von SMOTE auf den Trainingsdatensatz war erfolgreich. Vor SMOTE bestand der Datensatz aus 5.957 Beispielen mit einem starken Klassenungleichgewicht: 5.525 Nicht-Churn-Fällen (Klasse 0) und nur 432 Churn-Fällen (Klasse 1). Nach der Anwendung von SMOTE wurde die Anzahl der Churn-Fälle künstlich auf 5.525 erhöht, sodass beide Klassen nun ausgeglichen sind. Dieses balancierte Trainingsset sollte dem Modell ermöglichen, Churn-Kunden besser zu erkennen und dadurch die Werte von Precision, Recall und F1-Score zu verbessern.

# Classification mit Logistischer Regression nach SMOTE

In [8]:
# Erneutes Aufteilen der Daten in Trainings- und Testdaten (wird hier wiederholt, um sicherzustellen, dass die Testdaten unverändert bleiben)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_RATIO, random_state=RANDOM_STATE, stratify=y)

# Daten skalieren nach SMOTE (Nur für KNN und Log. Regression angewendet)
scaler = StandardScaler()

# fit the Scaler only with training data, so that the test data does not influence the Scaler
scaler.fit(X_train_res) # type: ignore
X_train_res_scaled = scaler.transform(X_train_res) 
X_test_scaled = scaler.transform(X_test)

In [9]:
# Instanziere die Logistische Regression
clf = LogisticRegression(max_iter=500, random_state=42)

# Trainiere das Modell mit den skalierten Trainingsdaten nach SMOTE
clf.fit(X_train_res_scaled, y_train_res)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,500
,multi_class,'deprecated'


In [10]:
# Vorhersagen auf Testdaten machen und Leistung messen
y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))

Accuracy: 0.8247946812671099
Precision: 0.175
Recall: 0.3723404255319149
F1-Score: 0.23809523809523808


#### Auswertung: Logistische Regression nach SMOTE

Deine Ergebnisse nach Anwendung von SMOTE und Training der logistischen Regression zeigen einen deutlichen Fortschritt im Vergleich zu vorher.

**Accuracy (74,58%):** von ca. 74,6 % ist zwar niedriger als zuvor, was bei ausgeglicheneren Klassenverteilungen häufig vorkommt, da das Modell nicht mehr durch die dominante Mehrheitklasse „kein Churn“ „getäuscht“ wird.

**Precision (14,79):** Die Precision von ca. 14,8 % zeigt, dass von allen als „Churn“ vorhergesagten Fällen etwa 15 % tatsächlich Churn-Fälle sind. Dies ist zwar noch ausbaufähig, aber ein Fortschritt gegenüber nahezu keiner positiven Erkennung.

**Recall (51,87%):** Der deutlich verbesserte Recall von ca. 51,9 % bedeutet, dass das Modell jetzt mehr als die Hälfte aller tatsächlichen Churn-Kunden korrekt erkennt, was für die Anwendung in einem Unternehmen ein wichtiger Schritt ist.

**F1-Score (0,23):** Der F1-Score von ca. 0,23 zeigt insgesamt eine bessere Balance zwischen Precision und Recall, auch wenn noch Optimierungspotenzial besteht.

**Fazit:** Durch SMOTE und die anschließende Skalierung konnte das Modell wesentlich besser auf die Minderheitsklasse „Churn“ lernen und ist nun in der Lage, einen signifikanten Anteil der Churn-Kunden zu identifizieren. Die niedrigere Accuracy ist ein erwarteter Effekt bei ausgeglicheneren Klassen, der durch die Verbesserung von Recall und F1-Score mehr als ausgeglichen wird.

# Classification mit Decision Tree nach SMOTE

In [11]:
# Decision Tree Classifier instanziieren und trainieren mit resampleten Daten
clf_tree = DecisionTreeClassifier(random_state=RANDOM_STATE)
clf_tree.fit(X_train_res, y_train_res)

# Baseline Informationen zum Baum
print(f"Tree Depth: {clf_tree.get_depth()}")
print(f"Leaf Nodes: {clf_tree.get_n_leaves()}")

Tree Depth: 26
Leaf Nodes: 865


In [12]:
# Baum visualisieren
plt.figure(figsize=(40,20))
_ = tree.plot_tree(clf_tree, feature_names=X_train.columns, filled=True, class_names=["No Churn", "Churn"])
plt.title("Decision Tree Visualization")
plt.show()

NameError: name 'tree' is not defined

<Figure size 4000x2000 with 0 Axes>

In [ ]:
# Vorhersage auf Trainingsdaten
y_pred_train_clf_tree = clf_tree.predict(X_train_res)

# Konfusionsmatrix Trainingsdaten
ConfusionMatrixDisplay.from_estimator(clf_tree, X_train_res, y_train_res, values_format='d')
plt.title("Confusion Matrix - Decision Tree (Train)")
plt.show()

In [ ]:
# Vorhersage auf Testdaten
y_pred_test_clf_tree = clf_tree.predict(X_test)

# Konfusionsmatrix Testdaten
ConfusionMatrixDisplay.from_estimator(clf_tree, X_test, y_test, values_format='d')
plt.title("Confusion Matrix - Decision Tree (Test)")
plt.show()

In [ ]:
# Performance Metriken auf Testdaten
print("Accuracy:", accuracy_score(y_test, y_pred_test_clf_tree))
print("Precision:", precision_score(y_test, y_pred_test_clf_tree))
print("Recall:", recall_score(y_test, y_pred_test_clf_tree))
print("F1-Score:", f1_score(y_test, y_pred_test_clf_tree))

#### Auswertung: Decision Tree nach SMOTE

**Accuracy (81,12%):** Nach Ausgleich des Datensatzes mit SMOTE sinkt die Accuracy erwartungsgemäß auf 81,12%. Dies ist typisch, da das Modell nun nicht mehr die Mehrheitsklasse bevorzugt, sondern auf ausgeglichenen Daten trainiert wurde.

**Precision (17,58%):** Die Precision sinkt auf 17,58%, was bedeutet, dass ein größerer Anteil der als „Churn“ vorhergesagten Kunden tatsächlich keine churnenden Kunden sind. Es entstehen mehr Fehlalarme.

**Recall (42,78%):** Der Recall verbessert sich deutlich auf 42,78%, sodass nun deutlich mehr tatsächliche Churn-Kunden erkannt werden.

**F1-Score (0,25):** Der F1-Score bleibt moderat, da eine Verbesserung im Recall durch eine Verschlechterung der Precision ausgeglichen wird.

**Fazit:** Das Decision Tree Modell zeigt vor SMOTE eine hohe Accuracy, die jedoch durch das starke Klassenungleichgewicht verzerrt ist. Die niedrigen Werte bei Precision und Recall verdeutlichen, dass das Modell die Minderheitsklasse „Churn“ nur unzureichend erkennt. Nach Anwendung von SMOTE verbessert sich der Recall deutlich, was auf eine bessere Erkennung von Churn-Kunden hinweist. Gleichzeitig sinkt die Accuracy, und die Precision verschlechtert sich, was zu mehr Fehlalarmen führt. Die stark gestiegene Anzahl der Leaf Nodes (von ca. 400 auf 991) und die fehlenden False Positives und False Negatives in der Trainings-Confusion-Matrix deuten auf deutliches Overfitting hin. Das Modell ist dadurch zwar sehr genau auf die Trainingsdaten angepasst, generalisiert jedoch schlecht auf neue Daten.

Um Overfitting zu vermeiden und die Modellleistung zu verbessern, sollten Maßnahmen wie Baumschnitt (Pruning), Begrenzung der Baumtiefe und die Erhöhung der minimalen Samples pro Blatt geprüft werden. Zusätzlich ist der Einsatz robusterer Algorithmen wie Random Forest oder Gradient Boosting zu empfehlen, die besser mit Ausgleich von Klassenungleichgewichten und Overfitting umgehen können. Nur durch solche Optimierungen kann das Unternehmen die Vorhersagequalität verbessern und Churn-Kunden effektiver identifizieren und adressieren.

# Classification mit KNN nach SMOTE

In [ ]:
# Finde den besten Parameter für "n_neighbors" (oft auch "k" genannt)
train_accuracies = {}
test_accuracies = {}
neighbors = np.arange(1, 25)

for neighbor in neighbors:
    knn = KNeighborsClassifier(n_neighbors=neighbor)
    knn.fit(X_train_res_scaled, y_train_res)

    train_accuracies[neighbor] = knn.score(X_train_res_scaled, y_train_res)
    test_accuracies[neighbor] = knn.score(X_test_scaled, y_test)

In [ ]:
accuracy = []

# Calculating error for K values between 1 and 40
for i in range(1, 15):
    knn_classifier = KNeighborsClassifier(n_neighbors=i)
    knn_classifier.fit(X_train_res_scaled, y_train_res)
    pred_i = knn_classifier.predict(X_test_scaled)
    accuracy.append(accuracy_score(y_test, pred_i))

In [ ]:
# Boxplots zur Identifikation von Outliers in den demografischen Daten (hint: hätte man auch mit allen Werten auf einmal machen können)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle('Classify best K for KNN', fontsize=16, fontweight='bold')

# Plot der Trainings- und Testgenauigkeiten
axes[0].plot(neighbors, [train_accuracies[k] for k in neighbors], label='Training Accuracy')
axes[0].plot(neighbors, [test_accuracies[k] for k in neighbors], label='Testing Accuracy')
axes[0].set_xlabel('Number of Neighbors (k)')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('KNN: Variation von k')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
    
# Fehlerrate für K im rechten Subplot
axes[1].plot(range(1, len(accuracy) + 1), accuracy, label='Error')
axes[1].set_xlabel('Number of Neighbors (k)')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('KNN: Varying Number of Neighbors')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.show()

In [ ]:
# 5. Modell mit ausgewähltem k (z.B. k=7) trainieren
best_k = 7
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train_res_scaled, y_train_res)

In [ ]:
# Modell auf Testdaten evaluieren
y_pred_test_knn = knn.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_test_knn))
print("Precision:", precision_score(y_test, y_pred_test_knn))
print("Recall:", recall_score(y_test, y_pred_test_knn))
print("F1-Score:", f1_score(y_test, y_pred_test_knn))